### Extract Lyrics Embeddings

In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
import scipy as sp

/Users/chloe/opt/miniconda3/envs/BKMS/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import dataset
data = pd.read_csv('./spotify_data.csv', index_col=[0])
lyrics_data = pd.DataFrame(data['lyrics'])
lyrics_data['embedding'] = ''
lyrics_data['lyrics'] = lyrics_data['lyrics'].apply(lambda e: e.replace('"',''))

In [3]:
# define model
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig, pipeline
import torch

config = RobertaConfig.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
config.output_hidden_states = True
tok = RobertaTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
model = RobertaModel.from_pretrained("j-hartmann/emotion-english-distilroberta-base", config=config)

# extract embedding
lyrics_data['embedding'] = lyrics_data['lyrics'].apply(lambda e: np.array(model(torch.tensor([tok.encode(e)[:512]])).pooler_output.squeeze().detach()))

# lyrics_data['embedding'] = lyrics_data['lyrics'].apply(lambda e: model(torch.tensor([tok.encode(e)])).pooler_output.squeeze())
lyrics_data

Some weights of RobertaModel were not initialized from the model checkpoint at j-hartmann/emotion-english-distilroberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Token indices sequence length is longer than the specified maximum sequence length for this model (674 > 512). Running this sequence through the model will result in indexing errors


,lyrics,embedding
160,Aah Aah The games you played were never fun Yo...,"[0.11173111, 0.5557255, 0.8257218, 0.35561693,..."
445,The lights go out and I can't be saved Tides t...,"[0.010673863, 0.5355972, 0.69947267, 0.1620012..."
763,"'Cause you're a sky, 'cause you're a sky full ...","[0.26425898, 0.68403536, 0.20476605, 0.3347745..."
856,I will not make the same mistakes that you did...,"[-0.4957115, 0.19850674, 0.38352963, 0.2268695..."
1169,Heyy Heeey Heey Your lipstick stains On the fr...,"[0.54356205, 0.51118445, 0.26775378, 0.2068563..."
...,...,...
16203,"Yeah, you! Yeah, you! I used to wanna be Livin...","[0.5549803, 0.7041393, 0.281284, 0.30927595, -..."
16284,"If you don't wanna see me Did a full 180, craz...","[-0.027963923, 0.4214012, 0.027016114, 0.36592..."
16771,"Nah, nah, nah Cake by the ocean Oh, no See you...","[0.5492718, 0.5855223, 0.12780374, 0.22220916,..."
17046,"Come on, come on, turn the radio on It's Frida...","[0.3522463, 0.41048434, 0.06727339, 0.12759277..."


In [5]:
import pickle
with open('./lyrics_embedding.pkl', 'wb') as f:
    pickle.dump(lyrics_data, f)